<a href="https://colab.research.google.com/github/shiomasa1218/Colaboratory/blob/master/cnn5_jpynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
# drive mean root directory of  google drive
!ls ./gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/'train_folder_name'

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
ls: cannot access './gdrive/My Drive/Kumamoto-Univ/Graduationwork/exefolder/train_folder_name': Transport endpoint is not connected


In [3]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 17.7MB/s 


In [4]:
cd gdrive/'My Drive'/'Kumamoto-Univ'/'Graduationwork'/'exefolder'/

/content/gdrive/My Drive/Kumamoto-Univ/Graduationwork/exefolder


In [5]:
!ls

cnn5.py    input_data.py  __pycache__  test_folder_name
dft321.py  log.csv	  result.txt   train_folder_name


In [7]:
#CNNの実装

#python3の文法を2系で使うためのコマンド
#from __future__ import absolute_import, unicode_literals

import os
import random
import numpy as np
import csv
import pandas as pd
import input_data
import tensorflow as tf
import time
import datetime
from sklearn.metrics import confusion_matrix,recall_score,precision_score,f1_score
#import serial
import pandas as pd


#フィルタ作成
def weight_variable(shape):

    #initial = tf.truncated_normal(shape, stddev=np.sqrt(2.0/(shape[0]*shape[1]*shape[3])))
    initial = tf.truncated_normal(shape, stddev=0.1)
    # #initial = tf.random_uniform(minval= -1.0, maxval=1.0)
    return tf.Variable(initial)

def weight_variable_he(shape,nodes):
    initial = tf.truncated_normal(shape, stddev=np.sqrt(2.0/nodes))
    # initial = tf.random_uniform(minval= -1.0, maxval=1.0)
    return tf.Variable(initial)


#バイアス作成
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1,1,2,1], strides=[1,1,2,1],padding = 'VALID')

def batch_normalization(shape, input):
    eps = 1e-5
    gamma = weight_variable([shape])
    beta = weight_variable([shape])
    mean,variance = tf.nn.moments(input,[0])
    return gamma * (input - mean) / tf.sqrt(variance+eps) + beta

def Activation(x):
    datas = tf.nn.relu(x)
    #datas = tf.nn.tanh(x)
    return datas


#----------------
#  データの読み込み
#----------------


Data_Classes = 8 #学習クラス数
steps = 5000 #ミニバッチ学習回数
BATCH_SIZE = 1000 #ミニバッチの一回の学習で使う量
#TBATCH_SIZE = 10
# Validation = 0.8  #  ここの値で分割 ex. Validation = 0.4 →　0.4:0.6　に分割
CaptureNumber = 200 #ミニバッチの大きさ、初期は２００個のデータ
drop = 1.0 #dropoutの係数、NNのノードがこれをかけた数になる

allCSize = 4608 #全結合サイズ
filterSize = 64 #畳みこみフィルタ（カーネル）サイズ 今回は 1 x filiterSize の大きさ


# 訓練データのフォルダのパス
train_folder_name = "train_folder_name"
# 検証データのフォルダのパス
#test_folder_name = "test_folder_name"

# train_dir_name = ['pbldatas/d2','pbldatas/d3','pbldatas/d4']
# Ex_filename = ["carpet1","carpet2","carpet3","sponge_g","sponge_y","stonetile1","stonetile2","whitetile2","whitetile1","woodtile1"]

#全訓練データとそのラベル
All_Datas = []
All_Label = []

#外部検証データとそのラベル
Ex_TestDatas=[]
Ex_Label = []

#並び順をシャッフルしたあとの訓練データ
All_SDatas =[]
All_SLabel = []

#分割後の訓練データ，学習するほう
train_data = []
train_label = []

#分割後の訓練データ，モデル評価するほう
test_data = []
test_label = []

#フォルダ中身のファイル名を取得
Folder = os.listdir(train_folder_name)
# Folder2 = os.listdir(test_folder_name)


# .DS_Storeがあるか検知
dsflag = 0

if Folder[0] == ".DS_Store":
    Data_Classes = len(Folder)-1
    print("DS!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    dsflag = dsflag + 1
else:
    Data_Classes = len(Folder)

print(dsflag)
print(Data_Classes)
print(train_folder_name)

#フォルダごとにみる
#訓練データの読み込み
for i,d in enumerate(Folder):
    if d != ".DS_Store":
        #フォルダ内のファイルのリストを取得
        files = os.listdir(train_folder_name + '/'+d)
        print(files)

        # Ex_filename.append(d)

        #ファイル毎にみる
        for f in files:

            #.DS_Storeをどかす
            if f != ".DS_Store":
                #どのファイルを見ているか確認用
                #print("load:"+f)

                datafile_path = train_folder_name + '/' + d+'/'+f

                #csvから読み込み
                data = np.loadtxt(datafile_path, delimiter=",")     #pbldata用
                datalen = data[0,:].size

                #3x200のデータに整形
                x_csv = data[0, :]
                y_csv = data[1, :]
                z_csv = data[2, :]

                flatdata = []
                flatdata.append(x_csv)
                flatdata.append(y_csv)
                flatdata.append(z_csv)

                All_Datas.append(flatdata)

                #one_hot_vectorを作成し，ラベルとしてtrain_labelに追加
                tmp = np.zeros(Data_Classes)
                if dsflag == 0:
                    tmp[i] = 1
                else:
                    tmp[i-1] = 1
                All_Label.append(tmp)
            else:
                print(".ds_storeを除去")
    else:
        print("ds_storeを抹殺")




# 検証データの読み込み
# for i, d in enumerate(Folder2):
#     if d != ".DS_Store":
#         # フォルダ内のファイルのリストを取得
#         files = os.listdir(test_folder_name + '/' + d)
#         print(files)
#
#         # ファイル毎にみる
#
#         count1 = 0
#
#         for f in files:
#
#             # .DS_Storeをどかす
#             if f != ".DS_Store":
#                 # どのファイルを見ているか確認用
#                 # print("load:"+f)
#                 datafile_path = test_folder_name + '/' + d + '/' + f
#                 # csvから読み込み
#                 data = np.loadtxt(datafile_path, delimiter=",")     #pbldata用
#                 datalen = data[0, :].size
#                 # print(datalen)
#
#                 x_csv = data[0, :]
#                 y_csv = data[1, :]
#                 z_csv = data[2, :]
#
#                 x_200 = x_csv[400:600]
#                 y_200 = y_csv[400:600]
#                 z_200 = z_csv[400:600]
#
#                 flatdata = []
#                 flatdata.append(x_200)
#                 flatdata.append(y_200)
#                 flatdata.append(z_200)
#
#                 Ex_TestDatas.append(flatdata)
#
#                 tmp = np.zeros(Data_Classes)
#                 if dsflag == 0:
#                     print('on')
#                     print(count1)
#                     tmp[count1] = 1
#                     print(tmp)
#                 else:
#                     print('off')
#                     print(count1)
#                     tmp[count1 - 1] = 1
#                     print(tmp)
#
#                 Ex_Label.append(tmp)
#
#                 #緊急措置　挙動がわからないのでとりあえず
#                 count1 = count1+1
#
#
#             else:
#                 print(".ds_storeを除去")
#
#     else:
#         print("ds_storeを抹殺")
#
# print(Ex_TestDatas[0])
# print("Ex_Label")
# print(Ex_Label)

#訓練データ順列のシャッフル
np.random.seed(seed=32)
Rindex = np.random.permutation(list(range(len(All_Datas))))
print(Rindex)
for k in Rindex:
    All_SDatas.append(All_Datas[k])
    All_SLabel.append(All_Label[k])


#訓練データを学習に使うやつとモデルの評価に使うやつの２種に分ける
numberV = int(len(All_SDatas)*(0.9)) #訓練データ数
testV = len(All_SDatas) - numberV  #テストデータ数
train_data = All_SDatas[:numberV] #0.９まで
test_data = All_SDatas[numberV+1:]
train_label = All_SLabel[:numberV]
test_label = All_SLabel[numberV+1:]


print(len(train_data))
print(len(test_data))


#----------
#numpy行列へ
#----------
train_data = np.array(train_data)
train_label = np.array(train_label)

test_data = np.array(test_data)
test_label = np.array(test_label)

#------------
#placeholder
#-------------

sess = tf.InteractiveSession()
x = tf.placeholder('float', shape=[None,3,CaptureNumber]) #28x28の画像データ
y_ = tf.placeholder('float', shape=[None, Data_Classes]) #正解ラベル
keep_prob = tf.placeholder('float')

# -----------------------------------------
# 畳み込みニューラルネットワーク
#
# (convconv -> bn -> pool -> drop) x3，
# (fc->drop) x2
# -----------------------------------------

keep_prob = tf.placeholder('float')

#第1ブロック
W_conv1_1 = weight_variable([1,5,3,filterSize])
b_conv1_1 = bias_variable([filterSize])
x_image = tf.reshape(x,[-1,1,CaptureNumber,3])
h_conv1_1 = conv2d(x_image, W_conv1_1)
tan1_1 = Activation(h_conv1_1)
bn1_1 = batch_normalization(filterSize,tan1_1)

W_conv1_2 = weight_variable([1,5,filterSize,filterSize])
b_conv1_2 = bias_variable([filterSize])
h_conv1_2 = conv2d(bn1_1, W_conv1_2)
tan1_2 = Activation(h_conv1_2)
bn1_2 = batch_normalization(filterSize,tan1_2)

h_pool1 = max_pool_2x2(bn1_2)
h_pool1_drop = tf.nn.dropout(h_pool1, keep_prob)


# #第2ブロック
W_conv2_1 = weight_variable([1,5,filterSize,filterSize*2])
b_conv2_1 = bias_variable([filterSize*2])
h_conv2_1 = conv2d(h_pool1_drop, W_conv2_1)
tan2_1 = Activation(h_conv2_1)
bn2_1 = batch_normalization(filterSize*2,tan2_1)

W_conv2_2 = weight_variable([1,5,filterSize*2,filterSize*2])
b_conv2_2 = bias_variable([filterSize*2])
h_conv2_2 = conv2d(bn2_1, W_conv2_2)
tan2_2 = Activation(h_conv2_2)
bn2_2 = batch_normalization(filterSize*2,tan2_2)

h_pool2 = max_pool_2x2(bn2_2)
h_pool2_drop = tf.nn.dropout(h_pool2, keep_prob)

#

# #第3ブロック
W_conv3_1 = weight_variable([1,5,filterSize*2,filterSize*4])
b_conv3_1 = bias_variable([filterSize*4])
h_conv3_1 = conv2d(h_pool2_drop, W_conv3_1)
tan3_1 = Activation(h_conv3_1)
bn3_1 = batch_normalization(filterSize*4,tan3_1)

W_conv3_2 = weight_variable([1,5,filterSize*4,filterSize*4])
b_conv3_2 = bias_variable([filterSize*4])
h_conv3_2 = conv2d(bn3_1, W_conv3_2)
tan3_2 = Activation(h_conv3_2)
bn3_2 = batch_normalization(filterSize*4,tan3_2)


h_pool3 = max_pool_2x2(bn3_2)
h_pool3_drop = tf.nn.dropout(h_pool3, keep_prob)
h_pool3_flat = tf.reshape(h_pool3_drop, [-1,allCSize])

#全結合層
W_fc1 = weight_variable([allCSize, allCSize])
b_fc1 = bias_variable([allCSize])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)
bn4 = batch_normalization(allCSize,h_fc1)

# #dropout
h_fc1_drop = tf.nn.dropout(bn4, 1.0)

# #全結合層
W_fc2 = weight_variable([allCSize, allCSize])
b_fc2 = bias_variable([allCSize])

h_fc2= tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
bn5 = batch_normalization(allCSize,h_fc1)

# #dropout
h_fc2_drop = tf.nn.dropout(bn5, 0.5)

#softmaxで出力#
W_fc4 = weight_variable([allCSize, Data_Classes])
b_fc4 = bias_variable([Data_Classes])
y_conv=tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc4) + b_fc4)




#------------------
# 評価と損失関数
#------------------

#L2ノルムを足そうとした残骸
# L2_sqr = tf.nn.l2_loss(W_conv1) + tf.nn.l2_loss(W_conv2)+tf.nn.l2_loss(W_conv3) + tf.nn.l2_loss(W_fc1) + tf.nn.l2_loss(W_fc2)
# lambda_2 = 0.01

#クロスエントロピー誤差関数，1e-7を足して，勾配消失を防止
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv+1e-7),reduction_indices=[1]))
#loss = cross_entropy + lambda_2*L2_sqr
# cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0)))

#重みの最適化
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
#　推定結果のy_convと正解ラベルy_が同じかどうか判定，correct_predictionがbool配列になってる
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
# 正答率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# run
sess.run(tf.global_variables_initializer())



#
#学習器の保存処理
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state('./')
cwd = os.getcwd()

#モデルのロード
# if ckpt:
#      saver.restore(sess,cwd+"\\model.ckpt")
# #

#処理時間計測
startTime = datetime.datetime.today()

#ログ
learn = open('log.csv','w')
result = open('result.txt','w')


#learn.write("学習ログ データ: %s クラス数:%d ステップ数:%d\n\n"%(train_dir_name[0],Data_Classes,steps))
learn.write("class,step\n" )
learn.write("%g,%g\n"%(Data_Classes,steps))
learn.write("step,train_a,test_a\n" )
#result.write("結果，confusion matrix\n\n")

#学習ループ
#配列に入れた加速度データから，ランダムにデータを選択，さらにそこから3x200に切り出して学習にかける
for i in range(steps):

    #加速度データの中からランダムにデータを選択
    batch_mask = np.random.choice(len(train_data),BATCH_SIZE)
    tbatch_mask = np.random.choice(len(test_data),BATCH_SIZE)

    #バッチ配列
    train_data_batch = []
    train_label_batch = train_label[batch_mask]
    test_data_batch = []

    #3x200に切り出し_訓練データ
    for g in batch_mask:
        b_data = train_data[g]

        b_x = b_data[0]
        b_y = b_data[1]
        b_z = b_data[2]

        rdata = []
        start = random.randint(0, len(b_x) - CaptureNumber)
        end = start + CaptureNumber

        x_data = b_x[start:end]
        y_data = b_y[start:end]
        z_data = b_z[start:end]

        rdata.append(x_data)
        rdata.append(y_data)
        rdata.append(z_data)

        train_data_batch.append(rdata)

    # 3x200に切り出し_テストデータ
    for g in range(len(test_data)):
        b_data = test_data[g]

        b_x = b_data[0]
        b_y = b_data[1]
        b_z = b_data[2]

        rdata = []
        start = random.randint(0, len(b_x) - CaptureNumber)
        end = start + CaptureNumber

        x_data = b_x[start:end]
        y_data = b_y[start:end]
        z_data = b_z[start:end]

        rdata.append(x_data)
        rdata.append(y_data)
        rdata.append(z_data)

        test_data_batch.append(rdata)


    # 学習の実行　x＝データy＿＝ラベル keep_prob=dropoutでどれだけノード消すか
    w_out, _ = sess.run([y_conv, train_step],feed_dict={x: train_data_batch, y_: train_label_batch, keep_prob:drop})

    #精度の計算
    train_accuracy = accuracy.eval(feed_dict={
        x: train_data_batch, y_: train_label_batch, keep_prob:drop})

    # 10ステップごとに精度を記録
    if i % 10 == 0 :
        test_accuracy = accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop})

        print("step %d, training accuracy %g   test accuracy %g" % (i, train_accuracy, test_accuracy))
        learn.write("%d,%g,%g\n" % (i, train_accuracy, test_accuracy))

    #終了一回前にいろいろ作成
    if i == steps-1:

        # confusion matrix 作成
        y_p = tf.argmax(y_conv, 1)
        val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x: test_data_batch, y_: test_label, keep_prob: drop})
        y_true = np.argmax(test_label, 1)
        con = confusion_matrix(y_true, y_pred)
        print(con)

        #検証データ分類用
        #y_pred_1 = sess.run([y_p],feed_dict={x: Ex_TestDatas, keep_prob: drop})
        # y_pred2 = y_pred_1[0]
        # print(y_pred2)
        # print(len(y_pred2))



        print("ccn test 正答率 %g" % accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop
        }))

#         print("ccn realtime test 正答率 %g" % accuracy.eval(feed_dict={
#             x: Ex_TestDatas, y_: Ex_Label, keep_prob: drop
#         }))

        #処理時間計測，終わり
        endTime = datetime.datetime.today()

        print("")
        print("開始時刻 %s" % (startTime))
        print("終了時刻 %s" % (endTime))

        learn.write("\n開始時刻 %s\n" % (startTime))
        learn.write("終了時刻 %s\n" % (endTime))
        learn.write("ccn test 正答率 %g" % accuracy.eval(feed_dict={
            x: test_data_batch, y_: test_label, keep_prob: drop
        }))

        #confusion matrixについて，精度を百分率表記したcon1と何が何回正解したかを表すconの両方つくる
        con_sum = np.sum(con, axis=1)
        con1 = []
        for a in range(len(con)):
            c = con[a] / con_sum[a]
            con1.append(c)

        print(con_sum)
        print(con)
        result.write("Confusion Matrix :\n")
        # np.savetxt('result.txt',con,fmt="%0.2f")
        np.savetxt('confusion-no.csv', con, fmt="%0.3f", delimiter=',')
        np.savetxt('confusion.csv', con1, fmt="%0.3f", delimiter=',')


        ##学習器の保存処理２
saver.save(sess,cwd+"\\0model.ckpt")



learn.close()
#result.close()


0
9
train_folder_name
['tcarpet1_241.csv', 'tcarpet1_29.csv', 'tcarpet1_255.csv', 'tcarpet1_269.csv', 'tcarpet1_282.csv', 'tcarpet1_296.csv', 'tcarpet1_254.csv', 'tcarpet1_240.csv', 'tcarpet1_297.csv', 'tcarpet1_283.csv', 'tcarpet1_256.csv', 'tcarpet1_268.csv', 'tcarpet1_243.csv', 'tcarpet1_257.csv', 'tcarpet1_295.csv', 'tcarpet1_28.csv', 'tcarpet1_281.csv', 'tcarpet1_242.csv', 'tcarpet1_246.csv', 'tcarpet1_294.csv', 'tcarpet1_280.csv', 'tcarpet1_290.csv', 'tcarpet1_285.csv', 'tcarpet1_252.csv', 'tcarpet1_253.csv', 'tcarpet1_247.csv', 'tcarpet1_284.csv', 'tcarpet1_293.csv', 'tcarpet1_287.csv', 'tcarpet1_291.csv', 'tcarpet1_250.csv', 'tcarpet1_278.csv', 'tcarpet1_244.csv', 'tcarpet1_251.csv', 'tcarpet1_245.csv', 'tcarpet1_292.csv', 'tcarpet1_222.csv', 'tcarpet1_279.csv', 'tcarpet1_286.csv', 'tcarpet1_236.csv', 'tcarpet1_237.csv', 'tcarpet1_223.csv', 'tcarpet1_221.csv', 'tcarpet1_230.csv', 'tcarpet1_234.csv', 'tcarpet1_235.csv', 'tcarpet1_218.csv', 'tcarpet1_220.csv', 'tcarpet1_231.csv',

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, training accuracy 0.611   test accuracy 0.525
step 10, training accuracy 0.97   test accuracy 0.8625
step 20, training accuracy 0.983   test accuracy 0.7
step 30, training accuracy 0.978   test accuracy 0.75
step 40, training accuracy 0.984   test accuracy 0.8
step 50, training accuracy 0.985   test accuracy 0.875
step 60, training accuracy 0.977   test accuracy 0.8875
step 70, training accuracy 0.963   test accuracy 0.8625
step 80, training accuracy 0.975   test accuracy 0.9125
step 90, training accuracy 0.955   test accuracy 0.8375
step 100, training accuracy 0.976   test accuracy 0.875
step 110, training accuracy 0.984   test accuracy 0.8375
step 120, training accuracy 0.978   test accuracy 0.8375
step 130, training accuracy 0.985   test accuracy 0.7875
step 140, training accuracy 0.991   test accuracy 0.825
step 150, training accuracy 0.991   test accuracy 0.8125
step 160, training accuracy 0.982   test accuracy 0.9125
step 170, training accuracy 0.996   test accuracy 0.775